# Import libraries

In [1]:
import datetime as datetime
from feature_extraction.indicator_features import *
from feature_extraction.daily_features import *  
import matplotlib.pyplot as plt
from model import *
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import pytse_client as tse
from pytse_client.download import download_financial_indexes
import seaborn as sns
from sklearn.model_selection import train_test_split
import ta
from zigzag import peak_valley_pivots

# Import data

In [2]:
tse.download(
    symbols = ['فولاد'], 
    write_to_csv = True, 
    include_jdate = True
)
folad = tse.Ticker(
    symbol = 'فولاد',  
    adjust = True
)
foladHist = folad.history
foladClient = folad.client_types

print(f'Shape of Folad price history: {foladHist.shape}')
print(f"Shape of Folad's client history: {foladClient.shape}")

Shape of Folad price history: (3418, 10)
Shape of Folad's client history: (3061, 18)


In [3]:
print(f'{foladHist.info()}')
foladHist.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3418 entries, 0 to 3417
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       3418 non-null   datetime64[ns]
 1   open       3418 non-null   float64       
 2   high       3418 non-null   float64       
 3   low        3418 non-null   float64       
 4   adjClose   3418 non-null   float64       
 5   value      3418 non-null   int64         
 6   volume     3418 non-null   int64         
 7   count      3418 non-null   int64         
 8   yesterday  3418 non-null   float64       
 9   close      3418 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3)
memory usage: 267.2 KB
None


,date,open,high,low,adjClose,value,volume,count,yesterday,close
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0


In [4]:
print(f'{foladClient.info()}')
foladClient.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3061 entries, 0 to 3060
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         3061 non-null   object 
 1   individual_buy_count         3061 non-null   object 
 2   corporate_buy_count          3061 non-null   object 
 3   individual_sell_count        3061 non-null   object 
 4   corporate_sell_count         3061 non-null   object 
 5   individual_buy_vol           3061 non-null   object 
 6   corporate_buy_vol            3061 non-null   object 
 7   individual_sell_vol          3061 non-null   object 
 8   corporate_sell_vol           3061 non-null   object 
 9   individual_buy_value         3061 non-null   object 
 10  corporate_buy_value          3061 non-null   object 
 11  individual_sell_value        3061 non-null   object 
 12  corporate_sell_value         3061 non-null   object 
 13  individual_buy_mea

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,20221031,1825,9,1708,21,49802410,67671193,70802783,46670820,226547677883,307828826912,321904919165,212471585630,4548.930019,4546.500936,4548.890204,4552.557372,-21000373.0
1,20221030,1652,12,2414,34,50426966,140636184,106262442,84800708,226574515528,630754574388,476218265522,381110824394,4493.122103,4481.529471,4485.009167,4494.193897,-55835476.0
2,20221029,1638,21,2530,23,50846905,122975784,124985979,48836710,221414946403,536619789573,545694489320,212340246656,4354.541273,4366.045645,4363.621618,4347.963789,-74139074.0
3,20221026,2899,18,2360,18,66551328,88360803,131692816,23219315,284629534462,383228027171,567311554791,100546006842,4276.842296,4307.839805,4337.081762,4330.274465,-65141488.0
4,20221025,2790,13,2758,23,63159637,66522305,93539594,36142348,279337649018,295438134591,414270210601,160505573008,4422.724105,4428.821987,4441.189081,4440.928216,-30379957.0


In [5]:
foladClient = foladClient.iloc[::-1]
foladClient.reset_index(
    drop = True,
    inplace = True
)
foladClient['date'] = foladClient['date'].apply(
    lambda x: datetime.datetime.strptime(x, '%Y%m%d')
)
foladClient.head()

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,2008-11-26,14,2,4,1,106338,20010,126338,10,248937258,46844120,295757258,24120,2341.000000,2341.0,2341.035482,2412.000000,-20000.0
1,2008-11-29,116,175,3,2,4718258,33691903,121402,38288759,14980229312,107713006031,284080680,122409154663,3174.949168,2340.0,3196.999767,3196.999795,4596856.0
2,2008-11-30,10,2,1,1,87219,12791,100000,10,204005241,29918859,233900000,24100,2339.000000,2339.0,2339.055508,2410.000000,-12781.0
3,2008-12-01,8,1,5,1,128666,10,128666,10,300821108,24090,300821108,24090,2338.000000,2338.0,2409.000000,2409.000000,0.0
4,2008-12-02,7,1,1,0,18553,10000,28553,0,43358361,23370000,66728361,0,2337.000000,2337.0,2337.000000,NaN,-10000.0


In [6]:
marketIndex = download_financial_indexes(
    symbols = 'شاخص كل', 
    write_to_csv = True
)
marketIndex = marketIndex['شاخص كل']
marketIndex.rename(
    columns = {'value': 'TEDPIX'}, 
    inplace = True
)

print(f'Shape of TEDPIX: {marketIndex.shape}')
marketIndex.head()

Shape of TEDPIX: (3357, 2)


,TEDPIX,date
0,9248.9,2008-12-04
1,9248.9,2008-12-05
2,9178.3,2008-12-06
3,9130.5,2008-12-07
4,9089.2,2008-12-08


In [7]:
foladHist = pd.merge(
    foladHist, 
    marketIndex, 
    how = 'left', 
    on = 'date'
)
foladHist.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close,TEDPIX
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0,NaN
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0,NaN
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0,NaN
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0,NaN
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0,NaN


# Features

In [8]:
calCols = calender_features(foladHist)
candelCols = candlestick_feature(foladHist)
prpCols = prp_based(foladHist)
retCols = ret_based(foladHist)
shiftCols = shift_data(foladHist)
wghtCols = weight_feature(foladHist)

priceFeatures = pd.concat(
    [
        foladHist,
        calCols,
        candelCols,
        prpCols,
        retCols,
        shiftCols,
        wghtCols
    ], axis = 1
)

print(f'Shape of price features: {priceFeatures.shape}')
priceFeatures.head()

Shape of price features: (3418, 111)


,date,open,high,low,adjClose,value,volume,count,yesterday,close,TEDPIX,year,month,quarter,day,weekday,shadow_up7d,shadow_low7d,body7d,shadow_up14d,shadow_low14d,body14d,shadow_up21d,shadow_low21d,body21d,shadow_up30d,shadow_low30d,body30d,shadow_up60d,shadow_low60d,body60d,shadow_up90d,shadow_low90d,body90d,max_price7d,min_price7d,ave_price7d,first_price7d,last_price7d,yesterday_price7d,max_price14d,min_price14d,first_price14d,last_price14d,yesterday_price14d,max_price21d,min_price21d,first_price21d,last_price21d,yesterday_price21d,max_price30d,min_price30d,first_price30d,last_price30d,yesterday_price30d,max_price60d,min_price60d,first_price60d,last_price60d,yesterday_price60d,max_price90d,min_price90d,first_price90d,last_price90d,yesterday_price90d,prp_high30d,prp_high60d,prp_high90d,prp_low30d,prp_low60d,prp_low90d,prp_value3d30d,prp_value5d60d,ret1d_log,ret3d_log,ret7d_log,ret14d_log,ret30d_log,ret60d_log,lastclose_log,buy_queue_locked,sell_queue_locked,last1d_open,last1d_high,last1d_low,last1d_adjClose,last1d_value,last1d_volume,last1d_count,last1d_yesterday,last1d_close,last1d_TEDPIX,value,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0,NaN,1385,12,4,20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,889437216900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0,NaN,1385,12,4,21,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,0,16.0,16.0,16.0,16.0,8.894372e+11,468077431.0,7736.0,16.0,16.0,NaN,193879458000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0,NaN,1385,12,4,22,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0,0,16.0,16.0,16.0,16.0,1.938795e+11,100041000.0,9214.0,16.0,16.0,NaN,249241504527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0,NaN,1385,12,4,23,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0,0,16.0,16.0,16.0,16.0,2.492415e+11,126270939.0,5862.0,16.0,16.0,NaN,51666379451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0,NaN,1385,12,4,26,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,NaN,16.0,16.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0,0,16.0,16.0,16.0,16.0,5.166638e+10,26705128.0,1901.0,16.0,16.0,NaN,28239006789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
bbCols = BB(priceFeatures)
emaCols = EMA(foladHist)
macdCols = MACD(foladHist)
smaCols = SMA(foladHist)
stochasticCols = STOCHASTIC(foladHist)
rsiCols = RSI(foladHist)

indicatorFeatures = pd.concat(
    [
        bbCols,
        emaCols,
        macdCols,
        smaCols,
        stochasticCols,
        rsiCols,
    ], axis = 1
)

print(f'Shape of indicator features: {indicatorFeatures.shape}')
indicatorFeatures.head()

Shape of indicator features: (3418, 35)


,BB_bbh,BB_bbl,last_BB_bbh,last_BB_bbl,BB_buy,BB_position,EMA_5d,last_EMA_5d,EMA_40d,last_EMA_40d,EMA_buy,EMA_position,MACD,MACD_diff,MACD_signal,MACD_buy,MACD_position,SMA10d,SMA20d,last_SMA20d,SMA30d,SMA50d,last_SMA50d,SMA80d,SMA_buy,SMA_position,STOCH_fast,STOCH_slow,last_STOCH_fast,last_STOCH_slow,STOCH_buy,STOCH_position,RSI,RSI_buy,RSI_position
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
features = pd.concat(
    [
        priceFeatures,
        indicatorFeatures
    ],
    axis = 1
)
print(f'shape of features after concatenation: {features.shape}')

features.dropna(
    axis = 'columns', 
    thresh = len(features) - 100,
    inplace = True
)
print(f'shape of features after drop non-essential columns: {features.shape}')

features.dropna(
    axis = 'index', 
    how = 'any',
    inplace = True
)
print(f'shape of features after drop non-essential rows: {features.shape}')

features.head()

shape of features after concatenation: (3418, 146)
shape of features after drop non-essential columns: (3418, 136)
shape of features after drop non-essential rows: (3319, 136)


,date,open,high,low,adjClose,value,volume,count,yesterday,close,year,month,quarter,day,weekday,shadow_up7d,shadow_low7d,body7d,shadow_up14d,shadow_low14d,body14d,shadow_up21d,shadow_low21d,body21d,shadow_up30d,shadow_low30d,body30d,shadow_up60d,shadow_low60d,body60d,shadow_up90d,shadow_low90d,body90d,max_price7d,min_price7d,ave_price7d,first_price7d,last_price7d,yesterday_price7d,max_price14d,min_price14d,first_price14d,last_price14d,yesterday_price14d,max_price21d,min_price21d,first_price21d,last_price21d,yesterday_price21d,max_price30d,min_price30d,first_price30d,last_price30d,yesterday_price30d,max_price60d,min_price60d,first_price60d,last_price60d,yesterday_price60d,max_price90d,min_price90d,first_price90d,last_price90d,yesterday_price90d,prp_high30d,prp_high60d,prp_high90d,prp_low30d,prp_low60d,prp_low90d,prp_value3d30d,prp_value5d60d,ret1d_log,ret3d_log,ret7d_log,ret14d_log,ret30d_log,ret60d_log,lastclose_log,buy_queue_locked,sell_queue_locked,last1d_open,last1d_high,last1d_low,last1d_adjClose,last1d_value,last1d_volume,last1d_count,last1d_yesterday,last1d_close,value,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d,BB_bbh,BB_bbl,last_BB_bbh,last_BB_bbl,EMA_5d,last_EMA_5d,EMA_40d,last_EMA_40d,EMA_position,MACD,MACD_diff,MACD_signal,MACD_position,SMA10d,SMA20d,last_SMA20d,SMA30d,SMA50d,last_SMA50d,SMA80d,STOCH_fast,STOCH_slow,last_STOCH_fast,last_STOCH_slow,STOCH_position,RSI,RSI_position
99,2007-08-19,24.0,24.0,23.0,23.0,5885518656,2038628,455,23.0,23.0,1386,5,2,28,1,0.043478,0.000000,0.000000,0.045455,0.000000,0.045455,0.05,0.000000,0.150000,0.052632,0.000000,0.210526,0.066667,-0.533333,0.466667,0.066667,-0.466667,0.533333,24.0,23.0,17.300000,23.0,23.0,23.0,24.0,22.0,22.0,23.0,22.0,24.0,20.0,20.0,23.0,20.0,24.0,19.0,19.0,23.0,19.0,24.0,8.0,16.0,23.0,15.0,24.0,8.0,15.0,23.0,15.0,0.958333,0.958333,0.958333,1.916667,2.875,2.875,0.491480,0.407508,0.00000,0.00000,0.044452,0.139762,0.302281,0.427444,0.0,0,0,23.0,23.0,23.0,23.0,8.427706e+10,29760995.0,2983.0,23.0,23.0,5885518656,3.343660e+10,6.240610e+10,5.205234e+10,6.011786e+10,5.040312e+10,4.396833e+10,8.450347e+10,7.556703e+10,8.256046e+10,3.343660e+10,6.240610e+10,5.205234e+10,6.011786e+10,5.040312e+10,4.396833e+10,8.450347e+10,7.556703e+10,8.256046e+10,24.117100,18.982900,23.961250,18.838750,22.924505,22.886757,20.034374,19.882291,1.0,1.305622,-0.006286,1.311908,1.0,22.7,21.55,21.40,20.500000,18.62,18.46,17.3625,75.0,95.0,100.0,100.0,1.0,95.603490,0.0
100,2007-08-20,24.0,24.0,24.0,24.0,4439879570,1528358,605,23.0,24.0,1386,5,2,29,2,0.000000,0.000000,0.043478,0.000000,0.000000,0.090909,0.00,0.000000,0.200000,0.000000,0.000000,0.263158,0.000000,-0.533333,0.533333,0.000000,-0.500000,0.500000,24.0,23.0,17.428571,23.0,24.0,23.0,24.0,22.0,22.0,24.0,22.0,24.0,20.0,20.0,24.0,20.0,24.0,19.0,19.0,24.0,19.0,24.0,8.0,16.0,24.0,15.0,24.0,8.0,16.0,24.0,16.0,1.000000,1.000000,1.000000,2.000000,3.000,3.000,0.510145,0.387404,0.04256,0.04256,0.042560,0.182322,0.344840,0.470004,0.0,1,0,24.0,24.0,23.0,23.0,5.885519e+09,2038628.0,455.0,23.0,23.0,4439879570,2.860673e+10,6.181408e+10,5.153790e+10,6.010895e+10,5.042972e+10,4.388042e+10,8.451555e+10,7.559921e+10,7.371049e+10,2.860673e+10,6.181408e+10,5.153790e+10,6.010895e+10,5.042972e+10,4.388042e+10,8.451555e+10,7.559921e+10,7.371049e+10,24.423948,19.076052,24.117100,18.982900,23.283003,22.924505,20.227820,20.034374,1.0,1.340012,0.022483,1.317529,0.0,22.9,21.75,21.55,20.733333,18.80,18.62,17.4750,100.0,95.0,75.0,95.0,1.0,96.894196,0.0
101,2007-08-21,24.0,24.0,24.0,24.0,3514710100,1203668,303,24.0,24.0,1386,5,2,30,3,0.000000,0.000000,0.043478,0.000000,-0.045455,0.045455,0.00,0.000000,0.150000,0.000000,-0.052632,0.210526,0.000000,-0.533333,0.533333,0.000000,-0.500000,0.500000,24.0,23.0,17.557143,23.0,24.0,23.0,24.0,22.0,23.0,24.0,22.0,24.0,21.0,21.0,24.0,20.0,24.0,19.0,20.0,24.0,19.0,24.0,8.0,16.0,24.0,15.0,24.0,8.

In [26]:
pivots = pd.DataFrame(
    peak_valley_pivots(
        foladHist['adjClose'], 
        0.075, 
        -0.075
    ) * -1,
    columns = ['label']
)
signals = pivots.replace(
    to_replace = 0, 
    value = np.nan
)
signals.fillna(
    method = 'ffill', 
    inplace = True
)
# signals.replace(
#     to_replace = -1, 
#     value = 0,  
#     inplace = True
# )

np.unique(signals, return_counts = True)

(array([-1.,  1.]), array([1531, 1887], dtype=int64))

In [30]:
features

,date,open,high,low,adjClose,value,volume,count,yesterday,close,year,month,quarter,day,weekday,shadow_up7d,shadow_low7d,body7d,shadow_up14d,shadow_low14d,body14d,shadow_up21d,shadow_low21d,body21d,shadow_up30d,shadow_low30d,body30d,shadow_up60d,shadow_low60d,body60d,shadow_up90d,shadow_low90d,body90d,max_price7d,min_price7d,ave_price7d,first_price7d,last_price7d,yesterday_price7d,max_price14d,min_price14d,first_price14d,last_price14d,yesterday_price14d,max_price21d,min_price21d,first_price21d,last_price21d,yesterday_price21d,max_price30d,min_price30d,first_price30d,last_price30d,yesterday_price30d,max_price60d,min_price60d,first_price60d,last_price60d,yesterday_price60d,max_price90d,min_price90d,first_price90d,last_price90d,yesterday_price90d,prp_high30d,prp_high60d,prp_high90d,prp_low30d,prp_low60d,prp_low90d,prp_value3d30d,prp_value5d60d,ret1d_log,ret3d_log,ret7d_log,ret14d_log,ret30d_log,ret60d_log,lastclose_log,buy_queue_locked,sell_queue_locked,last1d_open,last1d_high,last1d_low,last1d_adjClose,last1d_value,last1d_volume,last1d_count,last1d_yesterday,last1d_close,value,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d,value_20d,value_30d,value_40d,value_50d,value_60d,value_70d,value_80d,value_90d,value_100d,BB_bbh,BB_bbl,last_BB_bbh,last_BB_bbl,EMA_5d,last_EMA_5d,EMA_40d,last_EMA_40d,EMA_position,MACD,MACD_diff,MACD_signal,MACD_position,SMA10d,SMA20d,last_SMA20d,SMA30d,SMA50d,last_SMA50d,SMA80d,STOCH_fast,STOCH_slow,last_STOCH_fast,last_STOCH_slow,STOCH_position,RSI,RSI_position
99,2007-08-19,24.0,24.0,23.0,23.0,5885518656,2038628,455,23.0,23.0,1386,5,2,28,1,0.043478,0.000000,0.000000,0.045455,0.000000,0.045455,0.050000,0.000000,0.150000,0.052632,0.000000,0.210526,0.066667,-0.533333,0.466667,0.066667,-0.466667,0.533333,24.0,23.0,17.300000,23.0,23.0,23.0,24.0,22.0,22.0,23.0,22.0,24.0,20.0,20.0,23.0,20.0,24.0,19.0,19.0,23.0,19.0,24.0,8.0,16.0,23.0,15.0,24.0,8.0,15.0,23.0,15.0,0.958333,0.958333,0.958333,1.916667,2.875000,2.875000,0.491480,0.407508,0.000000,0.000000,0.044452,0.139762,0.302281,0.427444,0.000000,0,0,23.0,23.0,23.0,23.0,8.427706e+10,29760995.0,2983.0,23.0,23.0,5885518656,3.343660e+10,6.240610e+10,5.205234e+10,6.011786e+10,5.040312e+10,4.396833e+10,8.450347e+10,7.556703e+10,8.256046e+10,3.343660e+10,6.240610e+10,5.205234e+10,6.011786e+10,5.040312e+10,4.396833e+10,8.450347e+10,7.556703e+10,8.256046e+10,24.117100,18.982900,23.961250,18.838750,22.924505,22.886757,20.034374,19.882291,1.0,1.305622,-0.006286,1.311908,1.0,22.7,21.55,21.40,20.500000,18.62,18.46,17.3625,75.000000,95.000000,100.000000,100.000000,1.0,95.603490,0.0
100,2007-08-20,24.0,24.0,24.0,24.0,4439879570,1528358,605,23.0,24.0,1386,5,2,29,2,0.000000,0.000000,0.043478,0.000000,0.000000,0.090909,0.000000,0.000000,0.200000,0.000000,0.000000,0.263158,0.000000,-0.533333,0.533333,0.000000,-0.500000,0.500000,24.0,23.0,17.428571,23.0,24.0,23.0,24.0,22.0,22.0,24.0,22.0,24.0,20.0,20.0,24.0,20.0,24.0,19.0,19.0,24.0,19.0,24.0,8.0,16.0,24.0,15.0,24.0,8.0,16.0,24.0,16.0,1.000000,1.000000,1.000000,2.000000,3.000000,3.000000,0.510145,0.387404,0.042560,0.042560,0.042560,0.182322,0.344840,0.470004,0.000000,1,0,24.0,24.0,23.0,23.0,5.885519e+09,2038628.0,455.0,23.0,23.0,4439879570,2.860673e+10,6.181408e+10,5.153790e+10,6.010895e+10,5.042972e+10,4.388042e+10,8.451555e+10,7.559921e+10,7.371049e+10,2.860673e+10,6.181408e+10,5.153790e+10,6.010895e+10,5.042972e+10,4.388042e+10,8.451555e+10,7.559921e+10,7.371049e+10,24.423948,19.076052,24.117100,18.982900,23.283003,22.924505,20.227820,20.034374,1.0,1.340012,0.022483,1.317529,0.0,22.9,21.75,21.55,20.733333,18.80,18.62,17.4750,100.000000,95.000000,75.000000,95.000000,1.0,96.894196,0.0
101,2007-08-21,24.0,24.0,24.0,24.0,3514710100,1203668,303,24.0,24.0,1386,5,2,30,3,0.000000,0.000000,0.043478,0.000000,-0.045455,0.045455,0.000000,0.000000,0.150000,0.000000,-0.052632,0.210526,0.000000,-0.533333,0.533333,0.000000,-0.500000,0.500000,24.0,23.0,17.557143,23.0,24.0,23.0,24.0,22.0,23.0,24.0,22.0,24.

In [33]:
initFeatures = pd.concat([features, signals[-len(features):]], axis = 1)
initFeatures.to_csv('initFeatures.csv', index = False)

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def rfcScore(
    n_estimators,
    min_samples_split,
    min_samples_leaf,
    max_depth):

    params = {
        'n_estimators': int(n_estimators),
        'criterion': 'entropy',
        'min_samples_split': int(min_samples_split),
        'min_samples_leaf': int(min_samples_leaf),
        'max_depth': int(max_depth),
        'max_features': 'sqrt',
        'n_jobs': -1,
        'random_state': 1,
        'class_weight': 'balanced_subsample'
    }

    rfc = RandomForestClassifier(
        **params
    )

    score = cross_val_score(
        rfc, 
        X = X_train,
        y = y_train,
        scoring = 'f1_weighted'
    ).mean()

    return score

rfcBound = {
    'n_estimators': (100, 200),
    'min_samples_split': (2, 5),
    'min_samples_leaf': (1, 5),
    'max_depth': (4, 8)
}

optimizer = BayesianOptimization(
f = rfcScore, 
pbounds = rfcBound,
random_state = 1
)
optimizer.maximize(
init_points = 2,
n_iter = 3,
)
# print(optimizer.max['params'])

In [ ]:
# import sklearn
# sklearn.metrics.get_scorer_names()

In [ ]:
y = signals[-len(features):].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(
    features.values, 
    y, 
    shuffle = False, 
    random_state = 0
)

# trainPred, testPred, trainProb, testProb, featureImport, trainClassReport,\
#     testClassReport = RFClf(X_train, y_train, X_test, y_test)

# print(f'Training classification report:\n{trainClassReport}')
# print(f'Testing classification report:\n{testClassReport}')

In [ ]:
featuresSize = len(features)
trainEnd = (-featuresSize + len(trainPred))

In [ ]:
pri_change = foladHist["adjClose"]  - foladHist["adjClose"].shift()
testPred_forChange.shift(2) * pri_change[trainEnd:]

In [ ]:
print(f'Passive invesitng: {((foladHist["adjClose"].iloc[-1] / foladHist["adjClose"].iloc[0]) -1 ):0.0%}')

In [ ]:
plt.figure(figsize = (25, 15))

plt.title('Price of Mobarakeh Steel Company', fontsize = 15)
plt.plot(
    foladHist.iloc[-featuresSize:]['date'], 
    np.log(foladHist.iloc[-featuresSize:]['adjClose']),
    color = 'blue',
    lw = 3,
    label = 'Price'
)
plt.plot(
    foladHist.iloc[-featuresSize:]['date'].iloc[(pivots !=0).values.ravel()[-featuresSize:]], 
    np.log(foladHist.iloc[-featuresSize:]['adjClose']).iloc[(pivots !=0).values.ravel()[-featuresSize:]],
    color = 'black',
    lw = 2,
    label = 'ZigZag'
)

# train
plt.scatter(
    foladHist.iloc[-featuresSize: trainEnd]['date'].iloc[trainPred == -1], 
    (np.log(foladHist.iloc[-featuresSize: trainEnd]['adjClose']).iloc[trainPred == -1]),
    color = 'red',
    s = 20,
    label = 'Sell Position based on RF (on train dataset)'
)
plt.scatter(
    foladHist.iloc[-featuresSize: trainEnd]['date'].iloc[trainPred == 1], 
    (np.log(foladHist.iloc[-featuresSize: trainEnd]['adjClose']).iloc[trainPred == 1]),
    color = 'green',
    s = 20,
    label = 'Buy Position based on RF (on train dataset)'
)

# test
plt.scatter(
    foladHist.iloc[trainEnd:]['date'].iloc[testPred == -1], 
    (np.log(foladHist.iloc[trainEnd:]['adjClose']).iloc[testPred == -1]),
    color = 'red',
    s = 70,
    alpha = 0.5,
    label = 'Sell Position based on RF (on test dataset)'
)
plt.scatter(
    foladHist.iloc[trainEnd:]['date'].iloc[testPred == 1], 
    (np.log(foladHist.iloc[trainEnd:]['adjClose']).iloc[testPred == 1]),
    color = 'green',
    s = 70,
    alpha = 0.5,
    label = 'Buy Position based on RF (on test dataset)'
)

plt.legend()